In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
rng = np.random.default_rng(seed = 1)

## Black Scholes

The Black-Scholes model for pricing European Call Options is as follows:

$C = N(d_1)S_t - N(d_2)Ke^{-rt}$ <br>
<br>
Where, <br>
$C=$ Call option price <br>
$N=$ CDF of the standard normal distribution <br>
$S_t=$ Spot price of the underlying asset <br>
$K=$ Strike price <br>
$r=$ Risk free rate <br>
$t=$ time to maturity <br>
$\sigma=$ Asset volatility

In [3]:
rng.normal(0,1,size=10)

array([ 0.34558419,  0.82161814,  0.33043708, -1.30315723,  0.90535587,
        0.44637457, -0.53695324,  0.5811181 ,  0.3645724 ,  0.2941325 ])

In [4]:
def black_scholes_call(S0: float, 
                       K: float, 
                       r: float, 
                       T: float, 
                       sigma: float):
    d1 = ((np.log(S0/K)+(r+ sigma **2 / 2)*(T))/(sigma * np.sqrt(T)))
    d2 = d1 - (sigma * np.sqrt(T))
    price = S0 * norm.cdf(d1, loc = 0, scale = 1) - \
            K * np.exp(-r * T) * norm.cdf(d2, loc = 0, scale = 1)
    
    return price

In [5]:
black_scholes_call(S0 = 50, 
                   K = 55,
                   r = 0.05,
                   T = 1,
                   sigma = 0.25)

4.013192346926676

Teaching note: maybe put some examples on the screen for them to calculate

## Binomial Model

The binomial model depends on a few variables:

$u = $ Change in the 'up' state <br>
$d = $ Change in the 'down' state (commonly assumed to be $d=1/u$) <br>
$p_u = $ Probability of the up state <br>
$p_d = 1- p_u =$ Probability of the down state <br>

Using these variables, a tree of prices (and payoffs) are created to value the option in question. In this course, we will be using the Cox Ross Rubinstein method, which assumes:

$u = e^{\sigma \sqrt{\Delta t}}$<br>
$d = 1/u = e^{-\sigma \sqrt{\Delta t}}$<br>
$p_u = \frac{e^{r\Delta t}-d}{u-d}$ <br>
$p_d = 1-p_u$<br>
<br>
Where:

$r=$ Risk free rate <br>
$t=$ Time to maturity <br>
$\Delta t =$ Change in time for each step <br>
$\sigma=$ Asset volatility <br>

We will now use this method to value a european call option.

In [6]:
# Define the payoff function for our option.
def call_value(S, K):
    return np.maximum(S-K, 0)

# Define the binomial model
def binomial_call_price(S0,r,K, sigma, N, T):
    u = np.exp(sigma * np.sqrt(T/N))
    d = 1/u
    pt = (np.exp(r*T/N)-d)/(u-d) # Calculate the probability of an upwards movement
    disc = np.exp(r*T) # Calculate the discount factor

    total = 0
    for i in range(0,N+1):
        x = math.comb(N,i) * (pt ** i) * ((1-pt)**(N-i)) * call_value(S0 * (u **i)*(d **(N-i)),K)
        total = total + x
    return 1/disc * total

In [13]:
# Now let's use the previous example, and include 252 nodes (i.e. a node per trading day)

binomial_call_price(S0 = 50,
                    r = 0.05,
                    K = 55,
                    sigma = 0.25,
                    N = 5,
                    T = 1)

# We can see that the resulting price is very similar to that of the black scholes model.

3.8683825375902283

Teaching note: get them to calc the same at various steps

## Monte Carlo

We now move onto the Monte Carlo method, which is the method we will be using throughout the rest of this topic and topic 4. Much like in topic 2, we begin our Monte Carlo by developing a function for price paths.

In [8]:
def random_walk(S0: float, T: float, sigma: float, mu: float, N: int, n_sims: int):
    # Define length of time steps
    dt = T/N
    # Calculate an array of price paths using GBM
    paths = S0 * np.exp(np.cumsum((mu - sigma**2/2)*dt + \
                                          sigma*np.sqrt(dt) * \
                                            rng.normal(size  = [N, n_sims]), axis = 0))
    return paths

In [9]:
# Now that we have a function for our price paths, we will have to calculate the payoff at the end of each path (as this is a European call).

def mc_euro_call(S0: float, T: float, sigma: float, r: float, N: int, n_sims: int, K: float):
    # First we generate our price paths
    paths = random_walk(S0, T, sigma, r, N, n_sims)
    # Now we calculate the payoffs
    payoffs = np.maximum(paths[-1] - K, 0)
    # Now we take the mean payoff and discount
    price = np.mean(payoffs) * np.exp(-r*T)
    return price

In [10]:
# Lets use this function to calculate the price of our call option.
mc_euro_call(S0 = 50,
          T = 1,
          sigma = 0.25,
          r = 0.05,
          N = 252,
          n_sims = 100_000,
          K = 55)

4.035145466710749

In [11]:
# We can also calculate the price of a put option by simply changing the payoff function.
def mc_euro_put(S0: float, T: float, sigma: float, r: float, N: int, n_sims: int, K: float) -> float:
    # First we generate our price paths
    paths = random_walk(S0, T, sigma, r, N, n_sims)
    
    # Now we calculate the payoffs
    payoffs = np.maximum(K - paths[-1], 0)

    # Now we take the mean payoff and discount
    price = np.mean(payoffs) * np.exp(-r*T)

    return price

In [12]:
mc_euro_put(S0 = 50,
         T = 1,
         sigma = 0.25,
         r = 0.05,
         N = 252,
         n_sims = 100_000,
         K = 45)

1.8812246811900843